In [1]:
# Deal with test data
import pandas as pd
from datetime import datetime

In [2]:
test_pd = pd.read_csv("ashrae-energy-prediction/test.csv", header='infer')
weather_test_pd = pd.read_csv("ashrae_intermidiate_result/weather_test_cld.csv", header='infer')
building_pd = pd.read_csv("ashrae_intermidiate_result/building_metadata_cld.csv", header='infer')

print(test_pd.shape)
print(weather_test_pd.shape)
print(building_pd.shape)

(41697600, 4)
(277243, 12)
(1449, 4)


In [3]:
test_pd.head()

,row_id,building_id,meter,timestamp
0,0,0,0,2017-01-01 00:00:00
1,1,1,0,2017-01-01 00:00:00
2,2,2,0,2017-01-01 00:00:00
3,3,3,0,2017-01-01 00:00:00
4,4,4,0,2017-01-01 00:00:00


In [4]:
weather_test_pd.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,timestamp_2,date,time
0,0,2017-01-01 00:00:00,17.8,4.0,11.7,0.0,1021.4,100.0,3.6,2017-01-01 00:00:00,2017-01-01,00:00:00
1,0,2017-01-01 01:00:00,17.8,2.0,12.8,0.0,1022.0,130.0,3.1,2017-01-01 01:00:00,2017-01-01,01:00:00
2,0,2017-01-01 02:00:00,16.1,0.0,12.8,0.0,1021.9,140.0,3.1,2017-01-01 02:00:00,2017-01-01,02:00:00
3,0,2017-01-01 03:00:00,17.2,0.0,13.3,0.0,1022.2,140.0,3.1,2017-01-01 03:00:00,2017-01-01,03:00:00
4,0,2017-01-01 04:00:00,16.7,2.0,13.3,0.0,1022.3,130.0,2.6,2017-01-01 04:00:00,2017-01-01,04:00:00


In [5]:
building_pd.head()

,site_id,building_id,primary_use,square_feet
0,0,0,Education,7432
1,0,1,Education,2720
2,0,2,Education,5376
3,0,3,Education,23685
4,0,4,Education,116607


In [6]:
#test_pd = test_pd.drop(columns=['datetime','timestamp'],axis=1)
weather_test_pd = weather_test_pd.drop(['precip_depth_1_hr','wind_direction','wind_speed','timestamp_2','date','time'],axis=1)

buidling_pd = building_pd.drop(columns=['primary_use'],axis=1)

test_building_pd = pd.merge(test_pd, buidling_pd, on = "building_id", how = "left")
merge_pd_test = pd.merge(test_building_pd, weather_test_pd, on=['site_id','timestamp'], how='left')


In [7]:
merge_pd_test.head()

,row_id,building_id,meter,timestamp,site_id,square_feet,air_temperature,cloud_coverage,dew_temperature,sea_level_pressure
0,0,0,0,2017-01-01 00:00:00,0,7432,17.8,4.0,11.7,1021.4
1,1,1,0,2017-01-01 00:00:00,0,2720,17.8,4.0,11.7,1021.4
2,2,2,0,2017-01-01 00:00:00,0,5376,17.8,4.0,11.7,1021.4
3,3,3,0,2017-01-01 00:00:00,0,23685,17.8,4.0,11.7,1021.4
4,4,4,0,2017-01-01 00:00:00,0,116607,17.8,4.0,11.7,1021.4


In [8]:
merge_pd_test = merge_pd_test.drop(['timestamp','row_id'],axis=1)
#merge_pd_test = merge_pd_test.drop(['row_id'],axis=1)
print(merge_pd_test.shape)


(41697600, 8)


In [9]:
print(merge_pd_test.isna().sum())
merge_pd_test = merge_pd_test.fillna(0)
print(merge_pd_test.isna().sum())

building_id                0
meter                      0
site_id                    0
square_feet                0
air_temperature       199029
cloud_coverage        199029
dew_temperature       199029
sea_level_pressure    199029
dtype: int64
building_id           0
meter                 0
site_id               0
square_feet           0
air_temperature       0
cloud_coverage        0
dew_temperature       0
sea_level_pressure    0
dtype: int64


In [11]:
merge_pd_test.to_csv('ashrae_intermidiate_result/merge_pd_test.csv')

In [13]:
train_df = pd.read_csv('ashrae_intermidiate_result/train_cld.csv', header="infer")
weather_train_df = pd.read_csv('ashrae_intermidiate_result/weather_train_cld.csv', header="infer")
building_df = pd.read_csv('ashrae_intermidiate_result/building_metadata_cld.csv', header="infer")

In [16]:
feature_1 = ['building_id','date','time','meter','meter_reading']
feature_2 = ['site_id','date','time','air_temperature','cloud_coverage','dew_temperature','sea_level_pressure']
feature_3 = ['building_id','site_id','square_feet']

train_df = train_df[feature_1]
weather_train_df = weather_train_df[feature_2]
building_df = building_df[feature_3]

train_building_df = pd.merge(train_df,building_df,on='building_id',how='left')
merge_df = pd.merge(train_building_df,weather_train_df,on=['site_id','date','time'],how='left')
print(merge_df.shape)

(20216100, 11)


In [19]:
merge_df = merge_df.drop(['date','time'],axis=1)
merge_df.head()

,building_id,meter,meter_reading,site_id,square_feet,air_temperature,cloud_coverage,dew_temperature,sea_level_pressure
0,0,0,0.0,0,7432,25.0,6.0,20.0,1019.7
1,1,0,0.0,0,2720,25.0,6.0,20.0,1019.7
2,2,0,0.0,0,5376,25.0,6.0,20.0,1019.7
3,3,0,0.0,0,23685,25.0,6.0,20.0,1019.7
4,4,0,0.0,0,116607,25.0,6.0,20.0,1019.7


In [21]:
print(merge_pd_test.isna().sum())
merge_pd_test = merge_df.fillna(0)
print(merge_pd_test.isna().sum())

building_id           0
meter                 0
site_id               0
square_feet           0
air_temperature       0
cloud_coverage        0
dew_temperature       0
sea_level_pressure    0
dtype: int64
building_id           0
meter                 0
meter_reading         0
site_id               0
square_feet           0
air_temperature       0
cloud_coverage        0
dew_temperature       0
sea_level_pressure    0
dtype: int64


In [22]:
merge_df.to_csv('ashrae_intermidiate_result/merge_pd.csv')